# 🧠 Transformando o RAG em Produto: API, WhatsApp


## 🔧 Criando uma API com FastAPI para consumir o RAG

In [ ]:
!pip install fastapi uvicorn

Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process
pip install fastapi uvicorn langchain langchain-chroma langchain-huggingface langchain-community sentence-transformers


In [ ]:
!pip install -U langchain langchain-community langchain-huggingface langchain-chroma

In [ ]:
from fastapi import FastAPI # A MAIS FÁCIL DE TODAS
from pydantic import BaseModel
from langchain_chroma import Chroma  # Atualizado
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint  # Atualizado
from langchain.chains import RetrievalQA
import os

app = FastAPI() # Classe que INSTANCIA o fast api

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma(
    persist_directory="./chroma_hf_db",
    embedding_function=embedding
)

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",  # <- ESSA LINHA É ESSENCIAL
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0.3
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

class Consulta(BaseModel): # Herdar
    pergunta: str
    

@app.post("/chat/") # Decorador
def responder(consulta: Consulta):
    resposta = qa.run(consulta.pergunta)
    return {"resposta": resposta}




Executar:
```bash
uvicorn main:app --reload


## 📲 WHATSAPP + IA: Como Integrar com Z-API e Twilio

### ✅ Z-API: Integração com WhatsApp via API Não Oficial

#### O que é Z-API?

Z-API é uma API não oficial do WhatsApp.

Ela funciona usando WhatsApp Web por trás dos panos, simulando o comportamento de um usuário real.

Ideal para projetos rápidos, testes e protótipos, com baixo custo.

#### Como funciona por trás?

Você escanear um QR Code com o WhatsApp Web.

A Z-API cria uma sessão conectada ao seu WhatsApp pessoal ou corporativo.

Sua aplicação então recebe e envia mensagens através dessa conexão.


#### 🛠️ Etapas para usar a Z-API

1. Crie uma conta gratuita:

- Acesse: https://z-api.io

- Faça login com e-mail e senha.


2. Crie uma nova instância:

- Clique em "Nova instância"

- Dê um nome, selecione um plano gratuito ou teste.


3. Escaneie o QR Code:

- Escaneie com seu celular na aba WhatsApp Web.

- Depois disso, sua instância ficará online e conectada ao seu número.


4. Pegue os dados da instância:

- Você verá dois dados importantes:

    - Instance ID
    - Token



#### 📤 Enviando mensagens via Z-API

Para enviar mensagem de volta, você faz um POST HTTP para:

```bash
POST https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text
```

Com o seguinte corpo:

```json 
{
  "phone": "5511999999999",
  "message": "Olá! Recebemos sua dúvida e responderemos em breve."
}



#### 📥 Recebendo mensagens com Webhook

- Z-API envia um POST automático para sua API quando o usuário responde.

- Exemplo de payload que sua API vai receber:

```json
{
  "message": "Olá, quero saber sobre reembolso",
  "phone": "5511999999999",
  "instanceId": "seu_instance",
  "timestamp": 1712271000
}
```



####  Integração com FastAPI:

In [ ]:

from fastapi import FastAPI, Request
import requests

app = FastAPI()

INSTANCE_ID = "sua_instance"
TOKEN = "seu_token"

@app.post("/whatsapp/")
async def receber_mensagem(request: Request):
    dados = await request.json()
    pergunta = dados.get("message")
    numero = dados.get("phone")

    # Aqui entra seu RAG:
    resposta = qa.run(pergunta)

    # Enviar a resposta de volta
    requests.post(
        f"https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text",
        json={
            "phone": numero,
            "message": resposta
        }
    )
    return {"ok": True}


#### 📦 Prós e Contras do Z-API:
![prosecontras](images/prosecontras.png)

### Fluxo da mensagem:

#### 📡 1. Cliente envia mensagem no WhatsApp
Imagine que o cliente digita:

> "Gostaria de saber sobre reembolso." 

Essa mensagem é enviada para o número WhatsApp conectado via Z-API (o seu número pessoal ou empresarial que foi escaneado no painel da Z-API).

#### 🔁 2. Z-API envia um `POST` para o seu webhook FastAPI

No painel da Z-API, você configurou um webhook assim:

``https://seuservidor.com/whatsapp/``

Quando a mensagem chega no WhatsApp, a Z-API chama sua rota /whatsapp/, enviando um JSON com os seguintes dados:

```json
{
  "message": "Gostaria de saber sobre reembolso.",
  "phone": "5511999999999"
}
```

#### 🧠 3. FastAPI recebe e processa no endpoint /whatsapp/

No seu código:

```python
@app.post("/whatsapp/")
async def receber_mensagem(request: Request):
    dados = await request.json()
    pergunta = dados.get("message")
    numero = dados.get("phone")
```

Você está capturando:

- `pergunta` → O conteúdo da mensagem.
- `numero` → O telefone de quem enviou.

#### 🤖 4. Essa pergunta é passada para o seu RAG (LangChain + Chroma + Hugging Face | Ollama )

```python 
resposta = qa.run(pergunta)
```

Aqui, o pipeline acontece:

1. `pergunta` → transformada em vetor.

2. Vetor → busca no Chroma pelos documentos mais próximos.

3. Os documentos recuperados → passados para o LLM (Mixtral via Hugging Face).

4. O modelo gera a resposta final, já com base na sua base de conhecimento.


#### 📤 5. A resposta é enviada de volta ao usuário no WhatsApp

Você faz uma chamada de `POST` para a Z-API:

```python
requests.post(
    f"https://api.z-api.io/instances/{INSTANCE_ID}/token/{TOKEN}/send-text",
    json={
        "phone": numero,
        "message": resposta
    }
)
```

✅ E o cliente recebe a resposta diretamente no WhatsApp, como se fosse uma pessoa respondendo.

#### RESUMO DO FLUXO VISUAL:
```csharp
[WhatsApp do Cliente]
        ↓
[Z-API Webhook]
        ↓
[FastAPI /whatsapp/] ← captura mensagem
        ↓
[RAG LangChain]
        ↓
[Busca no ChromaDB + LLM]
        ↓
[Resposta gerada]
        ↓
[Z-API envia ao WhatsApp]
        ↓
[Cliente recebe a resposta no WhatsApp]
```


### ✅ Twilio: Integração Oficial com o WhatsApp Business

Twilio é uma plataforma oficialmente aprovada pelo WhatsApp.

É usada por grandes empresas como Nubank, iFood, Mercado Livre.

Você precisa de um número WhatsApp Business verificado e aprovado.

#### 🛠️ Como funciona a integração via Twilio?

1. Criar conta no https://twilio.com/
Obtenha um SID da conta e um Token de autenticação.

2. Obtenha um número de WhatsApp Business sandbox
Vá para: Messaging > Try it out > WhatsApp sandbox

Siga as instruções para conectar seu WhatsApp ao sandbox.

3. Configure seu webhook de recebimento
Twilio enviará mensagens recebidas para um endpoint HTTP (como FastAPI).

#### 📤 Enviando mensagem via Twilio (Python)



In [ ]:
from twilio.rest import Client

account_sid = "SEU_SID"
auth_token = "SEU_TOKEN"
client = Client(account_sid, auth_token)

mensagem = client.messages.create(
    from_='whatsapp:+14155238886',
    body='Olá! Como posso ajudar?',
    to='whatsapp:+5511999999999'
)

print(mensagem.sid)


#### 📥 Recebendo mensagem com webhook (FastAPI)



In [ ]:
from fastapi import FastAPI, Form

app = FastAPI()

@app.post("/twilio/")
async def receber_mensagem(Body: str = Form(...), From: str = Form(...)):
    print(f"Mensagem de {From}: {Body}")
    resposta = qa.run(Body)

    # Aqui você pode usar Twilio para responder de volta
    # (igual ao código acima)
    return "ok"


#### 📦 Prós e Contras do Twilio:
![prosecontras2](images/prosecontras2.png)

# Agora vamos configurar uma forma solução controlada, local, escalável e com liberdade total. 

Vamos integrar seu RAG Python com um bot em Node.js via uma API HTTP local

## ✅ Passo a passo: rodando o llm + fastapi com python

### Criando ambiente virtual no python

```bash
python -m venv venv  
```
### Ativando o ambiente virtual

```bash
venv\Scripts\activate
```

### Bypass da excessão

Caso o erro abaixo apareca vc precisa pedir para que o Wondows ignore, apenas para aquele terminal, o bloqueio de execução de scripts.

![erro](erroo.png)

Execute o comando abaixo:
```bash
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process
```

Execute novamente o comando abaixo:
```bash
venv\Scripts\activate
```

Caso tenha conseguito seu terminal terá o nome da pasta do ambiente virtual como abaixo:

```bash
(venv) PS C:\Users\Eric\Desktop\aula> 
```

### Instale as dependências:

```bash
pip install -r requiremensts.txt
```

As dependencias do arquivo são as seguintes:

```
fastapi
uvicorn
pydantic
python-dotenv
langchain
langchain-chroma
langchain-ollama
```

### Inicializar o FAST-API:

```bash
uvicorn rag_api:app --host 0.0.0.0 --port 3000 --reload
```

O comando anterior inicializa o 'motor' uvicorn que permanece 'ligado' o tempo todo aguardando novas requisções.


## ✅ Passo a passo: rodando o bot.js com whatsapp-web.js

### 🧩 1. Instalar o Node.js

👉 Acesse:
https://nodejs.org/

- Baixe a versão LTS (recomendada)
- Instale com todas as opções padrão
- Após instalar, reinicie o terminal (ou VS Code)



### ✅ 2. Verifique se o Node e NPM foram instalados

```bash
node -v
npm -v
```

Se aparecerem versões como v18.x.x, 9.x.x, está tudo certo!


### 📁 3. Crie uma pasta para seu bot (caso ainda não tenha)

```bash
mkdir whatsapp-rag-bot
cd whatsapp-rag-bot
```



### 📦 4. Crie o projeto Node

```bash
npm init -y
```


### 📚 5. Instale as dependências

```bash
npm install whatsapp-web.js axios qrcode-terminal
```


### 📝 6. Crie o arquivo bot.js


```js
const { Client, LocalAuth } = require("whatsapp-web.js");
const axios = require("axios");
const qrcode = require("qrcode-terminal");

const client = new Client({
    authStrategy: new LocalAuth()
});

client.on("qr", (qr) => {
    console.log("📱 Escaneie o QR Code para conectar seu bot ao WhatsApp:");
    qrcode.generate(qr, { small: true });
});

client.on("ready", () => {
    console.log("✅ Bot do WhatsApp está pronto!");
});

client.on("message", async (message) => {
    if (message.isGroupMsg) return;
    if (!message.body) return;

    console.log(`📩 Mensagem recebida: ${message.body}`);

    try {
        const response = await axios.post("http://localhost:3000/chat", {
            prompt: message.body
        });

        const botResponse = response.data.response
            .replace(/<\/?[^>]+(>|$)/g, "")
            .replace(/\n/g, " ")
            .trim();

        await message.reply(botResponse);
        console.log(`🤖 Resposta enviada: ${botResponse}`);
    } catch (error) {
        console.error("❌ Erro ao processar a mensagem:", error.message);
        await message.reply("⚠️ Ocorreu um erro ao processar sua mensagem.");
    }
});

client.initialize();
```


### 🚀 7. Rode o bot

```bash
node bot.js
```

📱 Ele mostrará um QR Code no terminal → escaneie com seu WhatsApp Web.

### 🧠 8. Certifique-se de que sua API FastAPI (/chat) esteja instalada e rodando

```bash
pip install fastapi uvicorn langchain langchain-community langchain-chroma sentence-transformers python-dotenv requests pydantic
```


```bash
uvicorn rag_api:app --host 0.0.0.0 --port 3000 --reload
```
